# Notebook para el proyecto de predicción de atrasos de vuelos - HACKATHON ONE

## Data Engineer (DE)

Sección para las tareas de Data Engineer.

Encargado: Ismael Cerda

### Selección y Limpieza de Datos

## Feature Architect (FA)

Sección para las tareas de Feature Architect

Encargado: Eduardo Ayala

### Ingeniería de Atributos

## Machine Learning Engineer (MLE)

Sección para las tareas de Machine Learning Engineer

Encargado: Luis Jácome

### Entrenamiento y Evaluación base

## Machine Learning Operations (MLOps)

Sección para las tareas de Machine Learning Operations

Encargado: Nicolás Staffelbach

### Microservicio Python

## Data Analyst (DA)

Sección para las tareas de Data Analyst

Encargado: David Aragón

### Análisis de Datos Exploratorio EDA